<a href="https://colab.research.google.com/github/melanieyes/RNN/blob/main/RNN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf
import time

# Load and preprocess the dataset
def load_and_preprocess_data():
    train_dataset, val_dataset, test_dataset = tfds.load(
        name="imdb_reviews",
        split=('train[:80%]', 'train[80%:]', 'test'),
        as_supervised=True
    )

    # Convert TensorFlow Datasets to numpy arrays for processing
    def convert_to_numpy(dataset):
        data_numpy = list(tfds.as_numpy(dataset))
        X = tf.convert_to_tensor([text for text, _ in data_numpy], dtype=tf.string)
        y = np.array([label for _, label in data_numpy], dtype=np.int16)
        return X, y

    X_train, y_train = convert_to_numpy(train_dataset)
    X_val, y_val = convert_to_numpy(val_dataset)
    X_test, y_test = convert_to_numpy(test_dataset)

    return X_train, y_train, X_val, y_val, X_test, y_test

# Load data
X_train, y_train, X_val, y_val, X_test, y_test = load_and_preprocess_data()

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.LGQ21J_1.0.0/imdb_reviews-train.tfrecor…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.LGQ21J_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.LGQ21J_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
# Initialize TextVectorization
VOCAB_SIZE = 20000  # Increased to capture more unique words
text_encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)
text_encoder.adapt(X_train)

In [ ]:
# Build and compile LSTM model
def build_lstm_model():
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(1,), dtype=tf.string),
        text_encoder,
        tf.keras.layers.Embedding(input_dim=VOCAB_SIZE, output_dim=128, mask_zero=True),
        tf.keras.layers.LSTM(128, dropout=0.3),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(1e-4), metrics=['accuracy'])
    return model

# Train and evaluate LSTM model
print("\nTraining LSTM Model")
lstm_model = build_lstm_model()
start_time = time.time()
lstm_history = lstm_model.fit(
    X_train, y_train, epochs=10, batch_size=512,
    validation_data=(X_val, y_val),
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)]
)
end_time = time.time()
lstm_time = end_time - start_time
lstm_loss, lstm_accuracy = lstm_model.evaluate(X_test, y_test)

print(f'LSTM Model - Test Loss: {lstm_loss:.4f}, Test Accuracy: {lstm_accuracy:.4f}, Training Time: {lstm_time:.2f} seconds')


Training LSTM Model
Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 8s 157ms/step - accuracy: 0.5170 - loss: 0.6929 - val_accuracy: 0.5882 - val_loss: 0.6920
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 6s 147ms/step - accuracy: 0.5972 - loss: 0.6913 - val_accuracy: 0.6676 - val_loss: 0.6849
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 10s 141ms/step - accuracy: 0.6861 - loss: 0.6534 - val_accuracy: 0.7668 - val_loss: 0.5304
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 10s 146ms/step - accuracy: 0.8021 - loss: 0.4885 - val_accuracy: 0.8256 - val_loss: 0.4276
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 13s 205ms/step - accuracy: 0.8423 - loss: 0.4029 - val_accuracy: 0.8590 - val_loss: 0.3694
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 6s 146ms/step - accuracy: 0.8847 - loss: 0.3176 - val_accuracy: 0.8696 - val_loss: 0.3277
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 10s 139ms/step - accuracy: 0.9035 - loss: 0.2601 - val_accuracy: 0.8720 - val_loss: 0.3149
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 10s 141ms/step - accuracy: 0.9211 - loss:

In [ ]:
# Build and compile GRU model
def build_gru_model():
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(1,), dtype=tf.string),
        text_encoder,
        tf.keras.layers.Embedding(input_dim=VOCAB_SIZE, output_dim=128, mask_zero=True),
        tf.keras.layers.GRU(128, dropout=0.3),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(1e-4), metrics=['accuracy'])
    return model

# Train and evaluate GRU model
print("\nTraining GRU Model")
gru_model = build_gru_model()
start_time = time.time()
gru_history = gru_model.fit(
    X_train, y_train, epochs=10, batch_size=512,
    validation_data=(X_val, y_val),
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)]
)
end_time = time.time()
gru_time = end_time - start_time
gru_loss, gru_accuracy = gru_model.evaluate(X_test, y_test)

print(f'GRU Model - Test Loss: {gru_loss:.4f}, Test Accuracy: {gru_accuracy:.4f}, Training Time: {gru_time:.2f} seconds')



Training GRU Model
Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 148ms/step - accuracy: 0.5072 - loss: 0.6929 - val_accuracy: 0.5508 - val_loss: 0.6917
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 10s 138ms/step - accuracy: 0.5566 - loss: 0.6910 - val_accuracy: 0.5798 - val_loss: 0.6886
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 133ms/step - accuracy: 0.6159 - loss: 0.6851 - val_accuracy: 0.6218 - val_loss: 0.6757
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 141ms/step - accuracy: 0.6713 - loss: 0.6571 - val_accuracy: 0.7580 - val_loss: 0.5121
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 10s 146ms/step - accuracy: 0.7975 - loss: 0.4606 - val_accuracy: 0.8376 - val_loss: 0.3733
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 10s 133ms/step - accuracy: 0.8766 - loss: 0.3000 - val_accuracy: 0.8730 - val_loss: 0.3058
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 6s 138ms/step - accuracy: 0.9142 - loss: 0.2278 - val_accuracy: 0.8760 - val_loss: 0.3213
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 130ms/step - accuracy: 0.9304 - loss: 0

In [ ]:
# Build and compile Bidirectional GRU model
def build_bigru_model():
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(1,), dtype=tf.string),
        text_encoder,
        tf.keras.layers.Embedding(input_dim=VOCAB_SIZE, output_dim=128, mask_zero=True),
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, dropout=0.3)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(1e-4), metrics=['accuracy'])
    return model

# Train and evaluate Bidirectional GRU model
print("\nTraining Bidirectional GRU Model")
bigru_model = build_bigru_model()
start_time = time.time()
bigru_history = bigru_model.fit(
    X_train, y_train, epochs=10, batch_size=512,
    validation_data=(X_val, y_val),
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)]
)
end_time = time.time()
bigru_time = end_time - start_time
bigru_loss, bigru_accuracy = bigru_model.evaluate(X_test, y_test)

print(f'Bidirectional GRU Model - Test Loss: {bigru_loss:.4f}, Test Accuracy: {bigru_accuracy:.4f}, Training Time: {bigru_time:.2f} seconds')



Training Bidirectional GRU Model
Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 13s 259ms/step - accuracy: 0.5130 - loss: 0.6928 - val_accuracy: 0.5588 - val_loss: 0.6916
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 10s 245ms/step - accuracy: 0.5726 - loss: 0.6903 - val_accuracy: 0.6044 - val_loss: 0.6873
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 10s 243ms/step - accuracy: 0.6315 - loss: 0.6828 - val_accuracy: 0.6460 - val_loss: 0.6715
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 10s 246ms/step - accuracy: 0.6939 - loss: 0.6514 - val_accuracy: 0.7430 - val_loss: 0.5590
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 10s 240ms/step - accuracy: 0.8057 - loss: 0.4502 - val_accuracy: 0.8476 - val_loss: 0.3456
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 10s 238ms/step - accuracy: 0.8906 - loss: 0.2760 - val_accuracy: 0.8790 - val_loss: 0.2971
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 10s 243ms/step - accuracy: 0.9196 - loss: 0.2086 - val_accuracy: 0.8796 - val_loss: 0.2940
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 10s 244ms/step - accuracy

In [ ]:
# Build and compile Bidirectional LSTM model
def build_bilstm_model():
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(1,), dtype=tf.string),
        text_encoder,
        tf.keras.layers.Embedding(input_dim=VOCAB_SIZE, output_dim=128, mask_zero=True),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, dropout=0.3)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(1e-4), metrics=['accuracy'])
    return model

# Train and evaluate Bidirectional LSTM model
print("\nTraining Bidirectional LSTM Model")
bilstm_model = build_bilstm_model()
start_time = time.time()
bilstm_history = bilstm_model.fit(
    X_train, y_train, epochs=10, batch_size=512,
    validation_data=(X_val, y_val),
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)]
)
end_time = time.time()
bilstm_time = end_time - start_time
bilstm_loss, bilstm_accuracy = bilstm_model.evaluate(X_test, y_test)

print(f'Bidirectional LSTM Model - Test Loss: {bilstm_loss:.4f}, Test Accuracy: {bilstm_accuracy:.4f}, Training Time: {bilstm_time:.2f} seconds')



Training Bidirectional LSTM Model
Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 14s 282ms/step - accuracy: 0.5132 - loss: 0.6929 - val_accuracy: 0.5882 - val_loss: 0.6920
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 20s 261ms/step - accuracy: 0.6006 - loss: 0.6908 - val_accuracy: 0.6580 - val_loss: 0.6853
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 20s 261ms/step - accuracy: 0.6856 - loss: 0.6616 - val_accuracy: 0.7786 - val_loss: 0.4886
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 21s 262ms/step - accuracy: 0.8004 - loss: 0.4605 - val_accuracy: 0.8432 - val_loss: 0.3836
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 10s 257ms/step - accuracy: 0.8711 - loss: 0.3437 - val_accuracy: 0.8660 - val_loss: 0.3356
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 21s 260ms/step - accuracy: 0.8935 - loss: 0.2786 - val_accuracy: 0.8696 - val_loss: 0.3176
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 21s 260ms/step - accuracy: 0.9122 - loss: 0.2423 - val_accuracy: 0.8660 - val_loss: 0.3315
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 10s 261ms/step - accurac

In [ ]:
# Comparison of Models
print("\nComparison of Models:")
print(f"LSTM Model - Test Accuracy: {lstm_accuracy:.4f}, Training Time: {lstm_time:.2f} seconds")
print(f"GRU Model - Test Accuracy: {gru_accuracy:.4f}, Training Time: {gru_time:.2f} seconds")
print(f"Bidirectional GRU Model - Test Accuracy: {bigru_accuracy:.4f}, Training Time: {bigru_time:.2f} seconds")
print(f"Bidirectional LSTM Model - Test Accuracy: {bilstm_accuracy:.4f}, Training Time: {bilstm_time:.2f} seconds")


Comparison of Models:
LSTM Model - Test Accuracy: 0.8682, Training Time: 98.35 seconds
GRU Model - Test Accuracy: 0.8723, Training Time: 84.75 seconds
Bidirectional GRU Model - Test Accuracy: 0.8779, Training Time: 103.18 seconds
Bidirectional LSTM Model - Test Accuracy: 0.8676, Training Time: 187.65 seconds
